In [29]:
import pandas as pd
import yfinance as yf
import gspread
from gspread_dataframe import get_as_dataframe

In [27]:
def get_open_positions_from_gsheet(sheet_name='myportfolio', worksheet_name='portfolio'):
    gc = gspread.service_account(filename='gdrive-creds.json')
    sh = gc.open(sheet_name)
    ws = sh.worksheet(worksheet_name)
    df = get_as_dataframe(ws)
    df = df.dropna(subset=['Symbol'])  # Clean up empty rows

    # If your columns sometimes have unnamed first column, clean it:
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])

    # Keep only the latest DateTime for each Symbol
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df = df.sort_values(['Symbol', 'DateTime'])
    latest_df = df.groupby('Symbol').tail(1).reset_index(drop=True)

    return latest_df


In [37]:
def format_symbol(raw_symbol, exchange):
    suffix = EXCHANGE_SUFFIX.get(exchange.upper())
    if suffix is None:
        raise ValueError(f"Unknown exchange: {exchange}")
    return raw_symbol + suffix


EXCHANGE_SUFFIX = {
    "NASDAQ": "",
    "NYSE": "",
    "AMEX": "",
    "LSE": ".L",
    "LSEETF": ".L",  # <-- support non-standard LSEETF label
    # add others like "ASX": ".AX", "PA": ".PA", etc.
}


In [39]:
port_df = get_open_positions_from_gsheet(sheet_name='myportfolio', 
                               worksheet_name='portfolio')



port_df["FormattedSymbol"] = port_df.apply(
    lambda r: format_symbol(r["Symbol"], r["Exchange"]), axis=1
)

In [41]:
port_df
unique_syms = port_df["FormattedSymbol"].unique().tolist()
unique_syms

['IB01.L', 'IBKR', 'IBTA.L', 'IGLN.L', 'IWDA.L', 'VUSD.L']

In [15]:
def get_latest_prices(symbols):
    price_dict = {}
    for symbol in symbols:
        try:
            data = yf.Ticker(symbol)
            latest = data.history(period="1d")['Close']
            if not latest.empty:
                price_dict[symbol] = latest.iloc[-1]
            else:
                price_dict[symbol] = None
        except Exception as e:
            price_dict[symbol] = None
    return price_dict


def get_latest_prices(symbols):
    price_dict = {}
    price_date_dict = {}
    for symbol in symbols:
        try:
            data = yf.Ticker(symbol)
            latest = data.history(period="1d")['Close']
            if not latest.empty:
                price_dict[symbol] = latest.iloc[-1]
                price_date_dict[symbol] = latest.index[-1].strftime('%Y-%m-%d')
            else:
                price_dict[symbol] = None
                price_date_dict[symbol] = None
        except Exception as e:
            price_dict[symbol] = None
            price_date_dict[symbol] = None
    return price_dict, price_date_dict


In [21]:
import yfinance as yf

etf = yf.Ticker("IBTA.L")  # or VUSD, VUSD.L, depending on listing
fi = getattr(etf, "fast_info", None)
fi.last_price

5.763999938964844

In [12]:
etf.history(period="1d")

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2025-06-23 00:00:00+01:00,112.922501,113.775009,112.762497,113.397499,760982,0.0,0.0,0.0


In [17]:
price_dict, price_date_dict = get_latest_prices(['IB01.L'])
price_dates = list(price_date_dict.values())
unique_dates = set([d for d in price_dates if d is not None])
if len(unique_dates) == 1:
    price_date = unique_dates.pop()
else:
    price_date = "Mixed"


price_date

'2025-06-23'

In [ ]:
price_dict, price_date_dict = get_latest_prices(['IB01.L'])

({'IBTA.L': np.float64(5.763999938964844)}, {'IBTA.L': '2025-06-23'})

In [24]:
EXCHANGE_SUFFIX = {
    "NASDAQ": "",
    "NYSE": "",
    "AMEX": "",
    "LSSETF": ".L",
    # add more as needed
}

def format_symbol(symbol:str, exchange:str):
    suffix = EXCHANGE_SUFFIX.get(exchange.upper())
    if suffix is None:
        raise ValueError(f"Unknown exchange: {exchange}")
    return f"{symbol}{suffix}"

format_symbol('IBTA','LSSETF')

'IBTA.L'

In [47]:
price_dict, price_date_dict = get_latest_prices(unique_syms)

In [48]:
price_dict

{'IB01.L': np.float64(116.23999786376953),
 'IBKR': np.float64(50.13759994506836),
 'IBTA.L': np.float64(5.763999938964844),
 'IGLN.L': np.float64(65.82499694824219),
 'IWDA.L': np.float64(113.91000366210938),
 'VUSD.L': np.float64(113.39749908447266)}

In [25]:
import yfinance as yf

EXCHANGE_SUFFIX = {
    "NASDAQ": "",
    "NYSE": "",
    "AMEX": "",
    "LSE": ".L",
}

def format_symbol(raw_symbol, exchange):
    suffix = EXCHANGE_SUFFIX.get(exchange.upper())
    if suffix is None:
        raise ValueError(f"Unknown exchange: {exchange}")
    return raw_symbol + suffix

def get_latest_prices(symbols):
    price_dict = {}
    price_date_dict = {}
    for symbol in symbols:
        try:
            data = yf.Ticker(symbol)
            latest = data.history(period="1d")["Close"]
            if not latest.empty:
                price_dict[symbol] = latest.iloc[-1]
                price_date_dict[symbol] = latest.index[-1].strftime("%Y-%m-%d")
            else:
                price_dict[symbol] = None
                price_date_dict[symbol] = None
        except Exception:
            price_dict[symbol] = None
            price_date_dict[symbol] = None
    return price_dict, price_date_dict

# Usage:
raw = [("AAPL", "NASDAQ"), ("IBTA", "LSE"), ("TSLA", "NYSE")]
symbols = [format_symbol(sym, exch) for sym, exch in raw]
prices, dates = get_latest_prices(symbols)
print(prices)
print(dates)


{'AAPL': np.float64(201.77499389648438), 'IBTA.L': np.float64(5.763999938964844), 'TSLA': np.float64(351.260009765625)}
{'AAPL': '2025-06-23', 'IBTA.L': '2025-06-23', 'TSLA': '2025-06-23'}


In [45]:
def calculate_yfinance_pnl(df, price_dict):
    """Given your positions DataFrame and a price_dict from get_latest_prices,
    calculate P&L columns."""
    results = []
    for _, row in df.iterrows():
        symbol = row['Symbol']
        qty = float(row['Position'])
        avg_cost = float(row['Average Cost'])
        last_price = price_dict.get(symbol)
        if last_price is not None:
            unrealized = (last_price - avg_cost) * qty
            pct = 100 * (last_price - avg_cost) / avg_cost if avg_cost != 0 else 0
            results.append({
                'Symbol': symbol,
                'Qty': qty,
                'Avg Cost': avg_cost,
                'Last Price': last_price,
                'Unrealized PnL': unrealized,
                'PnL %': pct
            })
    return pd.DataFrame(results)

# USAGE:
df = get_open_positions_from_gsheet()
#price_dict = get_latest_prices(['IBKR'])
price_dict = get_latest_prices(unique_syms)
pnl_df = calculate_yfinance_pnl(df, price_dict)
# display(pnl_df)
pnl_df

AttributeError: 'tuple' object has no attribute 'get'

In [ ]:
calculate_yfinance_pnl(df, price_dict)

In [14]:
def summarize_portfolio(df):
    summary_lines = []
    summary_lines.append(f"Portfolio as of: {df['DateTime'].max()}\n")
    summary_lines.append("Symbol  | Qty    | Avg Cost | Price   | Unrlz PnL | Rlz PnL")
    summary_lines.append("-" * 58)
    for _, row in df.iterrows():
        summary_lines.append(
            f"{row['Symbol']:<7} | {row['Position']:>7.4f} | "
            f"{row['Average Cost']:>8.2f} | {row['Market Price']:>7.2f} | "
            f"{row['Unrealized PnL']:>9.2f} | {row['Realized PnL']:>7.2f}"
        )
    summary_lines.append("-" * 58)
    summary_lines.append(f"TOTAL Unrealized PnL: ${df['Unrealized PnL'].sum():.2f}")
    summary_lines.append(f"TOTAL Realized PnL:   ${df['Realized PnL'].sum():.2f}")
    return "\n".join(summary_lines)

if __name__ == "__main__":
    df = get_open_positions_from_gsheet()
    msg = summarize_portfolio(df)
    print(msg)


Portfolio as of: 2025-06-22 21:26:22

Symbol  | Qty    | Avg Cost | Price   | Unrlz PnL | Rlz PnL
----------------------------------------------------------
IBKR    |  1.9276 |    51.71 |   51.53 |     -0.34 |    0.00
----------------------------------------------------------
TOTAL Unrealized PnL: $-0.34
TOTAL Realized PnL:   $0.00


In [26]:
def summarize_yfinance_portfolio(pnl_df):
    summary_lines = []
    summary_lines.append("Portfolio (latest market prices)\n")
    summary_lines.append("Symbol  | Qty    | Avg Cost | Price   | Unrlz PnL | PnL %  ")
    summary_lines.append("-" * 60)
    for _, row in pnl_df.iterrows():
        summary_lines.append(
            f"{row['Symbol']:<7} | {row['Qty']:>7.4f} | "
            f"{row['Avg Cost']:>8.2f} | {row['Last Price']:>7.2f} | "
            f"{row['Unrealized PnL']:>9.2f} | {row['PnL %']:>6.2f}%"
        )
    summary_lines.append("-" * 60)
    total_unrlz_pnl = pnl_df['Unrealized PnL'].sum()
    total_invested = (pnl_df['Avg Cost'] * pnl_df['Qty']).sum()
    total_pnl_pct = 100 * total_unrlz_pnl / total_invested if total_invested != 0 else 0
    summary_lines.append(f"TOTAL Portfolio Unrealized P&L: ${total_unrlz_pnl:.2f}")
    summary_lines.append(f"TOTAL Portfolio Unrealised P&L %: {total_pnl_pct:.2f}%")
    return "\n".join(summary_lines)

# Example usage:
msg = summarize_yfinance_portfolio(pnl_df)
print(msg)


Portfolio (latest market prices)

Symbol  | Qty    | Avg Cost | Price   | Unrlz PnL | PnL %  
------------------------------------------------------------
IBKR    |  1.9276 |    51.71 |   51.34 |     -0.71 |  -0.71%
------------------------------------------------------------
TOTAL Portfolio Unrealized P&L: $-0.71
TOTAL Portfolio Unrealised P&L %: -0.71%


In [15]:
import requests

def send_telegram_message(token, chat_id, message):
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message,
        "parse_mode": "Markdown"  # or "HTML"
    }
    response = requests.post(url, data=payload)
    if response.status_code == 200:
        print("Telegram message sent!")
    else:
        print("Failed to send:", response.text)


In [25]:
import requests

def send_telegram_message(token, chat_id, message):
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message,
        "parse_mode": "Markdown"  # or "HTML"
    }
    response = requests.post(url, data=payload)
    if response.status_code == 200:
        print("Telegram message sent!")
    else:
        print("Failed to send:", response.text)

if __name__ == "__main__":
    # Your portfolio summary message
    msg = """
Portfolio as of: 2025-06-22 21:26:22

Symbol  | Qty    | Avg Cost | Price   | Unrlz PnL | Rlz PnL
----------------------------------------------------------
IBKR    |  1.9276 |    51.71 |   51.53 |     -0.34 |    0.00
----------------------------------------------------------
TOTAL Unrealized PnL: $-0.34
TOTAL Realized PnL:   $0.00
"""
    TELEGRAM_BOT_TOKEN = "7973671527:AAERsWVREF8c1cTjCNsrZ7003xHviwmCzWc"
    TELEGRAM_CHAT_ID = "7311840033"

    send_telegram_message(TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID, msg)


Telegram message sent!


In [27]:
def get_account_summary_from_gsheet(sheet_name='myportfolio', worksheet_name='accountsummary'):
    gc = gspread.service_account(filename='gdrive-creds.json')
    sh = gc.open(sheet_name)
    ws = sh.worksheet(worksheet_name)
    df = get_as_dataframe(ws)
    df = df.dropna(subset=['Tag'])
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df = df.sort_values(['Tag', 'DateTime'])
    latest_df = df.groupby('Tag').tail(1).reset_index(drop=True)
    return latest_df


In [31]:
df = get_account_summary_from_gsheet(sheet_name='myportfolio', worksheet_name='accountsummary')

In [30]:
def summarize_account_summary(df):
    lines = []
    latest_datetime = df['DateTime'].max()
    lines.append(f"Account Summary as of: {latest_datetime}\n")
    lines.append("Tag                   | Value        | Currency")
    lines.append("-" * 45)
    for _, row in df.iterrows():
        lines.append(
            f"{row['Tag']:<22} | {row['Value']:>12} | {row['Currency']}"
        )
    lines.append("-" * 45)
    return "\n".join(lines)


In [33]:
print(summarize_account_summary(df))

Account Summary as of: 2025-06-22 21:26:22

Tag                   | Value        | Currency
---------------------------------------------
AvailableFunds         |       9965.0 | USD
BuyingPower            |       9965.0 | USD
NetLiquidation         |     10064.33 | USD
RealizedPnL            |          0.0 | BASE
TotalCashValue         |       9965.0 | USD
UnrealizedPnL          |        -0.34 | BASE
---------------------------------------------
